# Querying pdf with Astra and Langchain

Question Answering using Astra DB and Langchain,Powerd by Vector search

# Things to be done first
1. need serverless cassandra with vector search database on  Astra DB, create database and get the database token with role database administrator and copy your database ID: and these parameters need momentarily

2. we also need OpenAI API

### we will do
1. setup: import required deependaencies,provide secrets, create langchain vector store
2. Run a. question answering loop to get the relevant info with LLM

token"AstraCS:EfPdbCYxHgXcWPARfwuuhuMx:bbfdf6678c238a2516af0887e39e0daa62779f50f16aab8bbbfc492767976fc2"

In [ ]:
! pip install -q cassio datasets langchain openai tiktoken

In [6]:
# Langchain comp to be used
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

#for dataset retrievial from hugingface
from datasets import load_dataset_builder

#with cassio the engine powering the Astra DB integration in Langchain, we will aslo initialize the DB connection

import cassio


In [9]:
from PyPDF2 import PdfReader

# Required secrets
1. AstraDB connection details and
2. OpenAI API


In [7]:
from dotenv import load_dotenv
load_dotenv()  #as soon as we call it, it take environment variables from .env
import os



In [11]:
pdf_text=PdfReader('/Users/ravina/Desktop/langchainProjects/APJ Abdul Kalam Speech .pdf')

In [12]:
from typing_extensions import Concatenate
#read the text from given pdf
raw_text=""
for i, page in enumerate(pdf_text.pages):
  content=page.extract_text()
  if content:
    raw_text+=content

In [8]:
raw_text

'Thesis\nRavina\nMarch 2022\n1Contents\nAbstract\nAbbreviations and Notations\nIntroduction\nPreliminaries and Background\nAlgebraic Embedding\nContinuously differentiable Embedding\nSorting-based Embedding Schemes\nExperimental observations\nConclusion\n21 Abstract\nPermutation-invariant neural networks are gaining increasing popularity in Deep Learning. In this work, we\nare particularly interested in the permutation invariant neural networks which are injective. such networks\ncan recognize patterns in a set of inputs, even if the inputs are in different orders, without assuming that\nthe inputs have a fixed size.\nThese models can handle unordered input data and map each input to a unique output. This is particularly\nuseful in tasks such as set-to-set or set-to-sequence learning where the input and the output data are un-\nordered and unique mapping is required. However, the performance of these networks is highly sensitive to\ninput perturbation. A small perturbation in the input

# Initialize the connection to your database:


In [14]:
#cassio.init(token=os.getenv('ASTRA_DB_APP_TOKEN'),database_id=os.getenv('ASTRA_DB_ID'))

# create Langchain embeddings and llm objects

In [10]:
llm=OpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
embedding=OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

# create the langchain vector store backed by astra DB.

In [11]:
astra_vector_store=Cassandra(
    embedding=embedding,
    table_name="my_pdf_qna",
    session=None,
    keyspace=None
)

In [12]:
from langchain.text_splitter import CharacterTextSplitter
# using  CharacterTextSplitter to devide the text into chunk so that it shouldn't increase the token size

text_splitter=CharacterTextSplitter(
    separator='\n',
    chunk_size=700,
    chunk_overlap=200,
    length_function=len,
)

texts=text_splitter.split_text(raw_text)

In [ ]:
texts[:30] #top 30 chunks

# Load the dataset into vector store

In [14]:
astra_vector_store.add_texts(texts[:50])
print("inserted %i lines." %len(texts[:50]))
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

inserted 50 lines.


# Let's ask questions

In [15]:
first_question=True
while True:
  if first_question:
    query_text=input("\Enter your question (or type 'quit to exit):").strip()
  else:
    query_text=input("\what's your next question (or type 'quit to exit):").strip()

  if query_text.lower()=="quit":
    break

  if query_text.lower()=="":
    continue

  first_question=False

  print("\nQuestion: \%s\"" %query_text)
  answer=astra_vector_index.query(query_text,llm=llm).strip()
  print("answer:\'%s\'\n"%answer)

  print("First document by relevnce:")
  for doc,score in astra_vector_store.similarity_search_with_score(query_text,k=4):
    print("[%0.4f] \"%s ...\""% (score,doc.page_content[:100]))

\Enter your question (or type 'quit to exit):what is the topic of this document

Question: \what is the topic of this document"


answer:'The topic of this document is about permutation-invariant neural networks with injective properties and their applications in deep learning.'

First document by relevnce:


[0.8795] "Thesis
Ravina
March 2022
1Contents
Abstract
Abbreviations and Notations
Introduc ..."
[0.8724] "problem where we observed that the embedding dimension of the above map is relat ..."
[0.8682] "sub-matrix of A.
4. Improved bounds for the Bi-Lipschitz constant of the above e ..."
[0.8655] "the categorical or discrete data as continuous, high-dimensional vectors. We can ..."
\what's your next question (or type 'quit to exit):provide the 10 line summry of this document

Question: \provide the 10 line summry of this document"


answer:'This document discusses permutation-invariant neural networks and their use in Deep Learning. The focus is on injective networks that can recognize patterns in inputs regardless of their order or size. The problem of embedding dimension is addressed and it is shown that sorting combined with a linear mapping can reduce complexity. Polynomial embeddings and their algebraic equivalent description are also discussed. The Deep Sets architecture is explained, which uses a permutation-invariant function to classify sets of inputs. The importance of the injective property is emphasized.'

First document by relevnce:


[0.8663] "Thesis
Ravina
March 2022
1Contents
Abstract
Abbreviations and Notations
Introduc ..."
[0.8621] "problem where we observed that the embedding dimension of the above map is relat ..."
[0.8599] "In [1] polynomial embeddings (algebraic) are discussed by constructing their alg ..."
[0.8587] "the mean. This architecture ensures that the output of the network is invariant  ..."
\what's your next question (or type 'quit to exit):quit
